In [1]:
import os
import numpy as np
import tensorflow

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import MobileNet
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.optimizers import Adam

### Define the image size and batch size

In [3]:
image_size = 224
batch_size = 8

### Define the data directory

In [4]:
data_dir = "Dataset"

### Create an ImageDataGenerator for data augmentation

In [5]:
data_augmentation = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Split data into 80% training and 20% validation
)

### Load the training data

In [6]:
train_generator = data_augmentation.flow_from_directory(
    data_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 82 images belonging to 2 classes.


### Load the validation data

In [7]:
validation_generator = data_augmentation.flow_from_directory(
    data_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 20 images belonging to 2 classes.


### Build the MobileNet CNN model

In [8]:
base_model = MobileNet(input_shape=(image_size, image_size, 3), include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

17227776/17225924 [==============================] - 288s 17us/step


### Compile the model

In [9]:
optimizer = Adam(lr=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

### Train the model

In [11]:
epochs = 25  # You can increase the number of epochs for better accuracy
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

Epoch 1/25
10/10 [==============================] - 14s 1s/step - loss: 0.2875 - accuracy: 0.8919 - val_loss: 0.7732 - val_accuracy: 0.6875
Epoch 2/25
10/10 [==============================] - 14s 1s/step - loss: 0.4659 - accuracy: 0.8378 - val_loss: 1.3262 - val_accuracy: 0.5000
Epoch 3/25
10/10 [==============================] - 14s 1s/step - loss: 0.3372 - accuracy: 0.8649 - val_loss: 0.7036 - val_accuracy: 0.5625
Epoch 4/25
10/10 [==============================] - 14s 1s/step - loss: 0.2374 - accuracy: 0.8919 - val_loss: 1.9233 - val_accuracy: 0.5625
Epoch 5/25
10/10 [==============================] - 14s 1s/step - loss: 0.1436 - accuracy: 0.9324 - val_loss: 4.0754 - val_accuracy: 0.5625
Epoch 6/25
10/10 [==============================] - 14s 1s/step - loss: 0.2182 - accuracy: 0.9054 - val_loss: 2.6298 - val_accuracy: 0.6250
Epoch 7/25
10/10 [==============================] - 14s 1s/step - loss: 0.2375 - accuracy: 0.9189 - val_loss: 2.9513 - val_accuracy: 0.5625
Epoch 8/25
10/10 [==

### Save the model

In [12]:
model.save("keras_model.h5")

### Evaluate the model

In [13]:
scores = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print('Validation accuracy:', scores[1])

2/2 [==============================] - 1s 263ms/step - loss: 1.3431 - accuracy: 0.7500
Validation accuracy: 0.75


# THANK YOU!